In [2]:
import torch.optim as optim
from dataset import *
from model import *
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

train_log = np.array([])
test_log = np.array([])
lambda_l1 = 0

criterion = nn.L1Loss()
criterion = criterion.cuda()

model = VGG().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

tmp = np.load("../data/simu_20000_0.1_90_140_train.npy")
max = np.max(tmp[:, 1004])
min = np.min(tmp[:, 1004])
print(max, min)

train_dataset = Dataset("../data/simu_20000_0.1_90_140_train.npy", 0, 0)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataset = Dataset("../data/simu_10000_0.1_90_140_resonance_morlet.npy", 0, 1)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

for epoch in range(500):
    model.train()

    loss_total = 0
    step = 0
    
    loss_pure = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        output = model(data)
        
        loss = criterion(output, target)
        loss_pure = loss_pure + loss.item()
        
        l1_regularization = torch.tensor(0.0).cuda()
        for param in model.parameters():
            l1_regularization += torch.norm(param, p=1)
        loss += l1_regularization * lambda_l1
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_total = loss_total + loss.item()
        step = step + 1

    tmp = './pth/S_model_%d_%.4f.pth' % (epoch, loss_total/step)
    if epoch % 10 == 0:
        torch.save(model, tmp)
    print("S--epoch:" + str(epoch) + "    MAE:" + str(loss_total/step) + "   Pure:" + str(loss_pure/step))
    train_log = np.append(train_log, loss_total/step)

    loss_test = 0
    step = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.cuda(), target.cuda()
            output = model(data)
            print(output)

            # inv_norm
            # output = output * (max - min) + min

            loss = criterion(output, target)

            loss_test = loss_test + loss.item()
            step = step + 1

        loss_mean = loss_test / step
        print("epoch:" + str(epoch) + "    MAE_test:" + str(loss_mean))
        test_log = np.append(test_log, loss_mean)
    
    if epoch % 10 == 0 and epoch != 0:
        print("Test_epoch:" + str(epoch) + "    MAE_test:" + str(loss_mean))
        # tmp_epoch = np.arange(epoch+1)
        # print(tmp_epoch)
        # print(next(model.OneToOneLayer.parameters()))
        plt.plot(train_log)
        plt.plot(test_log)
        plt.ylim(0,20)
        plt.show()

140.0 90.0
S--epoch:0    MAE:11.001519789123535   Pure:11.001519789123535
tensor([[[114.1146]],

        [[105.8986]],

        [[112.5030]],

        [[118.6131]],

        [[102.5122]],

        [[118.3356]],

        [[111.1761]],

        [[107.8323]],

        [[124.7865]],

        [[130.6015]],

        [[100.8813]],

        [[116.0312]],

        [[104.4879]],

        [[111.9352]],

        [[118.0172]],

        [[104.9265]]], device='cuda:0')
tensor([[[113.2422]],

        [[128.1368]],

        [[110.7046]],

        [[106.2089]],

        [[119.1779]],

        [[108.5691]],

        [[113.2340]],

        [[111.6820]],

        [[131.0170]],

        [[124.4271]],

        [[114.0208]],

        [[102.9040]],

        [[ 93.2232]],

        [[111.0470]],

        [[105.4207]],

        [[116.2372]]], device='cuda:0')
tensor([[[101.9567]],

        [[115.1981]],

        [[105.6827]],

        [[105.5442]],

        [[117.9307]],

        [[119.5089]],

        [[101.5526

KeyboardInterrupt: 